In [128]:
# Input Parameters
R = 100  # cm
L = 80  # cm
f = 15  # cm
lam = 632.8e-8  # cm
lens_pos = 25+29.5+10 # cm

In [114]:
# Optical Matrix Elements
A = SecondaryValue('1-x/f', defaults=dict(f=f))
B = SecondaryValue('s+x-x*s/f', defaults=dict(f=f))
C = SecondaryValue('-1/f', defaults=dict(f=f))
D = SecondaryValue('1-s/f', defaults=dict(f=f))


In [161]:
w0 = SecondaryValue('((l/pi)^2*L*(R-L))', defaults=dict(l=lam, L=L, R=R, pi=np.pi))

In [139]:
# b parameter
b = SecondaryValue('l/(pi*w0^2)', defaults=dict(l=lam, pi=np.pi), dependencies=dict(w0=w0))
new_b = SecondaryValue('b*(A*D-C*B)/(A^2+B^2*b^2)', dependencies=dict(A=A, B=B, C=C, D=D, b=b))
b_before = SecondaryValue('b/(1+x^2*b^2)', dependencies=dict(b=b))

In [135]:
# omega
w = SecondaryValue('sqrt(l/(p*b))', defaults=dict(l=lam, p=np.pi), dependencies=dict(b=new_b))
w_before = SecondaryValue('sqrt(l/(p*b))', defaults=dict(l=lam, p=np.pi), dependencies=dict(b=b_before))

In [118]:
w0()

6.491621325058207e-09

In [140]:
data = pd.read_csv(r'./../messungen/Boettcher_Matthes/Kaustik-Messung/11_2008-07-13_03h-00m-41s.csv', sep=';', encoding='latin1')
fwhm = 'Value.18'
zs = np.array([11, 13, 15, 17, 19, 21, 23, 27])

In [141]:
def extract_fwhm(z):
    filename = glob.glob(f'./../messungen/Boettcher_Matthes/Kaustik-Messung/{z}*.csv')[0]
    dat = pd.read_csv(filename, sep=';', encoding='latin1')
    return dat[fwhm][0]

In [176]:
widths = np.array([extract_fwhm(z) for z in zs]) * 0.00056  # cm

In [178]:
popt, pcov = curve_fit(lambda x, w0: w(w0=w0, s=lens_pos, x=[x]), zs, widths)

In [200]:
%matplotlib qt5
xs_before = np.linspace(0, lens_pos, 1000)
xs_after = np.linspace(0, 40, 1000)
plt.plot(xs_before, w_before(w0=popt[0], x=[xs_before]), color='grey',
         linestyle='--')
plt.plot(lens_pos + xs_after, w(w0=popt[0], s=lens_pos, x=[xs_after]), color='grey',
         linestyle='--', label='Theoretische Kaustik, $w_0={:f}cm$'.format(popt[0]))
plt.errorbar(lens_pos + zs, widths, yerr=0.00056,
             xerr=1, linestyle='none', marker='x', markersize=10,
             ecolor='red', elinewidth=1, barsabove=True, label='Gemessene Kaustik')
plt.xlabel('z [cm]')
plt.ylabel('w [cm]')
plt.legend()
plt.show()

54.5

In [181]:
np.sqrt(np.diag(pcov))

array([0.00289796])

In [158]:
popt

array([0.05730852])

In [163]:
w0()

6.491621325058207e-09